In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH06-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Model


# ConversationBufferMemory

1. 개요
    - 대화 기록을 관리하는 메모리 클래스 중 하나로, 대화의 모든 기록을 순차적으로 저장하고 유지함
    - 이전 대화 내용을 기억하고 활용하여 대화의 문맥(Context)을 유지할 수 있음 

2. 주요 특징 
    1. 대화 기록 저장:
        - 사용자의 입력과 AI모델의 응답을 순서대로 저장 
        - 저장된 기록은 대화의 문맥을 이해하고, 이후 응답 생성에 사용
    2. 유지 시간:
        - 기본적으로 모든 대화 기록은 무기한 저장됨 
        - 대화 기록이 계속 누적되어 메모리 사용량이 증가할 수 있음 
        - 기록이 많아지면 프롬프트 크기가 증가하여 모델 호출 비용과 속도에 영향 미침 

[Reference] https://python.langchain.com/v0.1/docs/modules/memory/types/buffer/

In [3]:
from langchain.memory import ConversationBufferMemory

In [7]:
# 메모리 설정
memory = ConversationBufferMemory()
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

In [8]:
# 대화 저장
memory.save_context(
    inputs={
        "human": "안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

print(memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={})])


In [9]:
# 저장된 내용 확인
# `history` 키에 저장되어 있음
print(memory.load_memory_variables({})["history"])

Human: 안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?


추가 대화 #1

In [ ]:
# 대화 저장
memory.save_context(
    inputs={
        "human": "죄송한데 운전면허증은 없고, 모바일건감보험증은 있는데 가능한가요?"
    },
    outputs={
        "ai": "물론입니다. 모바일건강보험증도 가능합니다. 지금 화면에 보이시는 바코드 촬영해 주시면 본인인증 절차 진행하겠습니다."
    },
)

추가 대화 #2, #3 (순차적으로 실행/저장)

In [11]:
memory.save_context(
    inputs={"human": "네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. "},
    outputs={
        "ai": "감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)

memory.save_context(
    inputs={"human": "인증 번호 입력하여 완료 처리 하였습니다. "},
    outputs={
        "ai": "네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요."
    },
)

저장된 내용 확인

In [12]:
print(memory.load_memory_variables({})["history"])

Human: 안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?
Human: 죄송한데 운전면허증은 없고, 모바일건감보험증은 있는데 가능한가요?
AI: 물론입니다. 모바일건강보험증도 가능합니다. 지금 화면에 보이시는 바코드 촬영해 주시면 본인인증 절차 진행하겠습니다.
Human: 네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. 
AI: 감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다.
Human: 인증 번호 입력하여 완료 처리 하였습니다. 
AI: 네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요.


## 저장된 내용 객체 변환 

- 메모리에 저장된 내용은 텍스트 형태
- `return_messages=True` 로 설정하여 `HumanMessage` 와 `AIMessage` 객체로 반환

In [13]:
# 객체 변환 설정하여 초기화
memory = ConversationBufferMemory(return_messages=True)

memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True)

In [14]:
# 대화 내용 순차적으로 저장
memory.save_context(
    inputs={
        "human": "안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={
        "human": "죄송한데 운전면허증은 없고, 모바일건감보험증은 있는데 가능한가요?"
    },
    outputs={
        "ai": "물론입니다. 모바일건강보험증도 가능합니다. 지금 화면에 보이시는 바코드 촬영해 주시면 본인인증 절차 진행하겠습니다."
    },
)

memory.save_context(
    inputs={"human": "네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. "},
    outputs={
        "ai": "감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)

memory.save_context(
    inputs={"human": "인증 번호 입력하여 완료 처리 하였습니다. "},
    outputs={
        "ai": "네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요."
    },
)

In [15]:
# 대화 기록 확인
memory.load_memory_variables({})["history"]

[HumanMessage(content='안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='죄송한데 운전면허증은 없고, 모바일건감보험증은 있는데 가능한가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='물론입니다. 모바일건강보험증도 가능합니다. 지금 화면에 보이시는 바코드 촬영해 주시면 본인인증 절차 진행하겠습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='인증 번호 입력하여 완료 처리 하였습니다. ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요.', additional_kwargs={}, response_metadata={})]

## Chain 적용

In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

In [17]:
# 모델 정의/초기화
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
# 체인+메모리 구성
conversation = ConversationChain(
    llm=model,
    memory=memory,
)

In [21]:
# 대화 시작
# 메모리 저장된 첫번째 대화로 우선 시작한 경우
answers = conversation.predict(input="안녕하세요,은행 계좌를 재발급 하고 싶습니다.")

print(answers)

안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?


In [22]:
answers = conversation.predict(input="인증 번호 입력하여 본인 인증 완료 하였습니다. ")

print(answers)

인증이 완료되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요.


# ConversaionBufferWindowMemory

- `ConversaionBufferMemory`는 대화가 누적되어 계속 저장되어 메모리 사용량이 증가하는 단점 있음
- Window 메모리는 모든 대화 내용을 활용하는 것이 아닌 최근 k개 대화만 유지되도록 함

In [23]:
from langchain.memory import ConversationBufferWindowMemory

In [24]:
# 메모리 설정/초기화
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

/var/folders/37/2rwxb6k520l4tqxzpx1nh8lh0000gn/T/ipykernel_20335/3466125047.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, return_messages=True)


In [25]:
# 대화 저장
memory.save_context(
    inputs={
        "human": "안녕하세요, 은행 계좌를 재발급 하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 재발급 도와드리겠습니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)

memory.save_context(
    inputs={
        "human": "죄송한데 운전면허증은 없고, 모바일건감보험증은 있는데 가능한가요?"
    },
    outputs={
        "ai": "물론입니다. 모바일건강보험증도 가능합니다. 지금 화면에 보이시는 바코드 촬영해 주시면 본인인증 절차 진행하겠습니다."
    },
)

memory.save_context(
    inputs={"human": "네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. "},
    outputs={
        "ai": "감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다."
    },
)

memory.save_context(
    inputs={"human": "인증 번호 입력하여 완료 처리 하였습니다. "},
    outputs={
        "ai": "네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요."
    },
)

In [26]:
# 저장된 기록 확인
# 최근 2개만 저장하도록 설정하였음
memory.load_memory_variables({})["history"]

[HumanMessage(content='네, 바코드 통해 실행하여 확인 버튼 눌렀습니다. ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='감사합니다. 본인 인증 확인 되었습니다. 이제 휴대폰 인증 통한 본인 인증이 필요합니다. 문자로 발송된 인증번호를 입력해 주시면 됩니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='인증 번호 입력하여 완료 처리 하였습니다. ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요.', additional_kwargs={}, response_metadata={})]

In [27]:
# 체인+메모리 구성
conversation = ConversationChain(
    llm=model,
    memory=memory,
)

In [28]:
# 메모리에 저장된 내용으로 질의
answers = conversation.predict(input="인증 번호 입력하여 본인 인증 완료 하였습니다. ")

print(answers)

네! 본인 인증도 완료 되었습니다. 이제 원하시는 계좌 종류를 선택해 주세요.


In [29]:
# 메모리에 저장되지 않은 내용으로 질의
answers = conversation.predict(input="안녕하세요, 은행 계좌를 재발급 하고 싶습니다.")

print(answers)

안녕하세요! 은행 계좌를 재발급 받으시려면 본인 인증이 필요합니다. 이미 본인 인증을 완료하셨다면 어떤 종류의 계좌를 선택하시겠습니까? 예를 들어, 예금 계좌, 적금 계좌, 또는 기타 특별한 종류의 계좌가 있을 수 있습니다. 원하시는 계좌 종류를 알려주세요.


-----
** End of Documents **